In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

X = torch.tensor([[0,0], [0,1], [1,0], [1,1]], dtype=torch.float32)
y = torch.tensor([[0],   [1],   [1],   [0]],   dtype=torch.float32)

class XORModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(2, 2)
        
        # [수정됨] 활성화 함수: ReLU -> Sigmoid
        # 아까 수식에서 본 S자 곡선을 적용합니다.
        self.activation = nn.Sigmoid() 
        
        self.layer2 = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.layer1(x)
        x = self.activation(x) # 여기서 부드럽게 휜 공간이 만들어집니다.
        x = self.layer2(x)
        return self.sigmoid(x)

model = XORModel()

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(2000):
# [Step 1] 순전파 (Forward)
    # 모델에 X를 넣고 예측값(prediction)을 받아옵니다.
    prediction = model(X)
    
    # [Step 2] 오차 계산 (Loss Calculation)
    # 예측값과 정답(y)을 비교해 Loss(J)를 구합니다.
    loss = criterion(prediction, y)
    
    # [Step 3] 기울기 초기화 (Zero Grad) - 중요!
    # PyTorch는 기울기를 계속 누적하는 성질이 있습니다.
    # 이전 에폭의 찌꺼기 기울기를 0으로 깨끗이 지워줍니다.
    optimizer.zero_grad()
    
    # [Step 4] 역전파 (Backward) - 체인 룰 발동!
    # Loss에서부터 거꾸로 미분하며 각 파라미터(W)별 기울기를 계산합니다.
    # 계산된 기울기는 각 파라미터 내부에 저장됩니다.
    loss.backward()
    
    # [Step 5] 업데이트 (Step)
    # W_new = W_old - (lr * Gradient)
    # 계산된 기울기 방향으로 파라미터를 한 걸음 수정합니다.
    optimizer.step()

print("PyTorch (Sigmoid) 예측 결과:\n", model(X).detach().numpy())

PyTorch (Sigmoid) 예측 결과:
 [[0.01603173]
 [0.9808172 ]
 [0.9807922 ]
 [0.03328492]]


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

# 1. 데이터 준비 (XOR)
X = torch.tensor([[0,0], [0,1], [1,0], [1,1]], dtype=torch.float32)
y = torch.tensor([[0],   [1],   [1],   [0]],   dtype=torch.float32)

# 2. 모델 정의 (구조만 정의)
class XORModel(nn.Module):
    def __init__(self):
        super().__init__()
        # 가중치 초기값을 고정 (발표 때 항상 같은 결과가 나오도록)
        torch.manual_seed(42) 
        
        self.layer1 = nn.Linear(2, 2)  # 은닉층 (입력2 -> 은닉2)
        self.activation = nn.Sigmoid() # 활성화 (Sigmoid)
        self.layer2 = nn.Linear(2, 1)  # 출력층 (은닉2 -> 출력1)
        self.sigmoid = nn.Sigmoid()    # 최종 확률

# 모델 생성
model = XORModel()
criterion = nn.BCELoss()
lr = 0.1 # 학습률

print(f"{'='*20} [학습 시작 전 가중치(Weights)] {'='*20}")
print(f"Layer 1 Weights:\n{model.layer1.weight.data}")
print(f"Layer 2 Weights:\n{model.layer2.weight.data}\n")

# =================================================================
# ★ 여기가 핵심! (모델 내부를 한 단계씩 뜯어서 실행)
# =================================================================

print(f"{'='*20} [1. 순전파 (Forward Pass)] {'='*20}")
print("입력 데이터 X가 들어갑니다.\n")

# Step 1: 첫 번째 선형 변환 (z1 = W1*x + b1)
z1 = model.layer1(X)
# [중요] 중간 변수의 기울기를 기억하도록 설정 (원래는 버려짐)
z1.retain_grad() 
print(f"▶ Step 1 [Linear 1]: 입력 공간을 늘리고 회전시킴 (z1)\n{z1.detach().numpy()}\n")

# Step 2: 활성화 함수 (h1 = Sigmoid(z1))
h1 = model.activation(z1)
h1.retain_grad() # 기울기 기억
print(f"▶ Step 2 [Activation]: 공간을 부드럽게 휘게 만듦 (h1)\n{h1.detach().numpy()}\n")

# Step 3: 두 번째 선형 변환 (z2 = W2*h1 + b2)
z2 = model.layer2(h1)
z2.retain_grad() # 기울기 기억
print(f"▶ Step 3 [Linear 2]: 휘어진 공간에서 영역을 합침 (z2)\n{z2.detach().numpy()}\n")

# Step 4: 최종 출력 (y_pred = Sigmoid(z2))
y_pred = model.sigmoid(z2)
print(f"▶ Step 4 [Output]: 최종 확률값 예측 (y_pred)\n{y_pred.detach().numpy()}\n")


print(f"{'='*20} [2. 오차 계산 (Loss)] {'='*20}")
loss = criterion(y_pred, y)
print(f"Loss 값: {loss.item():.4f}")
print("(이 오차를 줄이기 위해 역전파를 시작합니다!)\n")


print(f"{'='*20} [3. 역전파 (Backward Pass)] {'='*20}")
# 역전파 실행! (모든 기울기가 계산됨)
loss.backward()

print("체인 룰(Chain Rule)을 타고 오차 신호(Gradient)가 거꾸로 흐릅니다.\n")

# Gradient 확인 (뒤에서부터)
print(f"◀ Gradient @ Output (dL/dy_pred): 결과가 얼마나 틀렸는지\n{z2.grad.numpy()}\n") # z2 입장에서의 변화량

print(f"◀ Gradient @ Layer 2 Weights (dL/dW2): 출력층 가중치를 얼마나 수정해야 할지")
print(f"{model.layer2.weight.grad.numpy()}\n")

print(f"◀ Gradient @ Hidden Layer (dL/dh1): 은닉층이 얼마나 잘못했는지")
print(f"{h1.grad.numpy()}\n") # 여기가 중요! 역전파가 은닉층까지 도달함

print(f"◀ Gradient @ Layer 1 Weights (dL/dW1): 은닉층 가중치를 얼마나 수정해야 할지")
print(f"{model.layer1.weight.grad.numpy()}\n")


print(f"{'='*20} [4. 파라미터 업데이트 (Update)] {'='*20}")
# 수동으로 업데이트 (W = W - lr * grad)
with torch.no_grad(): # 업데이트는 기록하지 않음
    model.layer1.weight -= lr * model.layer1.weight.grad
    model.layer2.weight -= lr * model.layer2.weight.grad

print("학습률(Learning Rate)을 곱해서 가중치를 수정했습니다.\n")
print(f"Updated Layer 1 Weights:\n{model.layer1.weight.data}")
print(f"Updated Layer 2 Weights:\n{model.layer2.weight.data}")

==================== [학습 시작 전 가중치(Weights)] ====================
Layer 1 Weights:
tensor([[ 0.5406,  0.5869],
        [-0.1657,  0.6496]])
Layer 2 Weights:
tensor([[-0.3443,  0.4153]])

==================== [1. 순전파 (Forward Pass)] ====================
입력 데이터 X가 들어갑니다.

▶ Step 1 [Linear 1]: 입력 공간을 늘리고 회전시킴 (z1)
[[-0.15492964  0.14268756]
 [ 0.4319746   0.7922438 ]
 [ 0.38568074 -0.02296811]
 [ 0.97258496  0.6265881 ]]

▶ Step 2 [Activation]: 공간을 부드럽게 휘게 만듦 (h1)
[[0.46134487 0.5356115 ]
 [0.60634506 0.68831295]
 [0.59524244 0.49425822]
 [0.7256344  0.65171546]]

▶ Step 3 [Linear 2]: 휘어진 공간에서 영역을 합침 (z2)
[[0.68694735]
 [0.7004423 ]
 [0.6236791 ]
 [0.6441781 ]]

▶ Step 4 [Output]: 최종 확률값 예측 (y_pred)
[[0.6652875 ]
 [0.66828585]
 [0.65105486]
 [0.6556973 ]]

==================== [2. 오차 계산 (Loss)] ====================
Loss 값: 0.7482
(이 오차를 줄이기 위해 역전파를 시작합니다!)

==================== [3. 역전파 (Backward Pass)] ====================
체인 룰(Chain Rule)을 타고 오차 신호(Gradient)가 거꾸로 흐릅니다.

◀ Gradient @ Outpu